In [5]:
import scanpy as sc
# import scvi
import glob
import os
from functools import reduce
import anndata as ad
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from scipy.io import mmread
import gzip
import dynamo as dyn
import velocyto as vcy

### Reading in loom from velocyto

In [6]:
vlm_pilot = vcy.VelocytoLoom("../velocyto_run//output/MR_30_10X_SA_PBS_BM_22112021_transcriptome_possorted_genome_bam_IFOE5.loom")
vlm_final = vcy.VelocytoLoom("../velocyto_run/output/MR_33_10X_SA_PBS_BM_transcriptome_possorted_genome_bam_IJA5R.loom")

### Configuring dynamo

In [8]:
dyn.configuration.set_figure_params('dynamo', background='white') # jupter notebooks
dyn.configuration.set_figure_params('dynamo', background='black') # presentation
# dyn.configuration.set_pub_style() # manuscript

In [9]:
# bm = sc.read("../output/bm_integrated.h5ad")
hvg = sc.read("../data/h5ad/hvg_integrated_170824.h5ad")

In [10]:
hvg.obs['pMito'] =hvg.obs['percent_mt']

In [12]:
hvg.obsm['X_scVI']

array([[ 1.3349354e-03,  4.7511292e-01,  4.7762804e-03, ...,
        -9.0266120e-01,  3.2771379e-02, -2.3049459e-02],
       [ 1.2936103e+00, -8.3951694e-01,  6.4081699e-03, ...,
         3.4301889e-01,  1.6294017e-02,  1.4192190e-02],
       [-4.7148120e-01,  4.1863799e-01, -1.0705569e-02, ...,
        -1.7194808e+00, -8.0057681e-03, -7.1350969e-03],
       ...,
       [-9.2762476e-01, -7.0137978e-02, -3.2809682e-02, ...,
         4.6381080e-01,  2.1407455e-02, -3.3722419e-02],
       [ 4.2715272e-01, -2.2141695e-02,  1.5500817e-02, ...,
        -7.6620609e-01,  2.0098925e-02, -3.7631989e-02],
       [-5.3124171e-01, -8.1957817e-02, -1.4697135e-02, ...,
        -5.5279493e-02,  3.4948431e-02,  2.8340647e-02]], dtype=float32)

In [13]:
dyn.pp.recipe_monocle(hvg)

|-----? dynamo.preprocessing.deprecated is deprecated.
|-----> recipe_monocle_keep_filtered_cells_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_cells_key=True
|-----> recipe_monocle_keep_filtered_genes_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_filtered_genes_key=True
|-----> recipe_monocle_keep_raw_layers_key is None. Using default value from DynamoAdataConfig: recipe_monocle_keep_raw_layers_key=True
|-----> apply Monocole recipe to adata...
|-----> ensure all cell and variable names unique.
|-----> ensure all data in different layers in csr sparse matrix format.


/tmp/ipykernel_529582/21529172.py:1: DeprecationWarning: recipe_monocle is deprecated and will be removed in a future release. Please update your code to use the new replacement function.
  dyn.pp.recipe_monocle(hvg)


|-----> ensure all labeling data properly collapased
|-----? dynamo detects your data is size factor normalized and/or log transformed. If this is not right, plese set `normalized = False.
|-----> filtering cells...
|-----> 5517 cells passed basic filters.
|-----> filtering gene...
|-----> 2391 genes passed basic filters.
|-----> calculating size factor...
|-----> selecting genes in layer: X, sort method: SVR...
|-----> size factor normalizing the data, followed by log1p transformation.
|-----> Set <adata.X> to normalized data
|-----> applying PCA ...
|-----> <insert> X_pca to obsm in AnnData Object.
|-----> cell cycle scoring...
|-----> computing cell phase...
|-----> [Cell Phase Estimation] completed [254.9905s]
|-----> [Cell Cycle Scores Estimation] completed [0.6273s]
|-----> [recipe_monocle preprocess] completed [3.5768s]


/home/anna/miniconda3/envs/dynamo/lib/python3.11/site-packages/dynamo/preprocessing/cell_cycle.py:460: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_cycle_scores["cell_cycle_order"] = cell_cycle_scores.groupby("cell_cycle_phase").cumcount()
/home/anna/miniconda3/envs/dynamo/lib/python3.11/site-packages/dynamo/preprocessing/cell_cycle.py:461: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cell_cycle_scores["cell_cycle_order"] = cell_cycle_scores.groupby("cell_cycle_phase", group_keys=False)[


In [14]:
hvg.layers

Layers with keys: counts, scVI_normalized, X_counts, X_scVI_normalized

In [15]:
hvg

AnnData object with n_obs × n_vars = 5517 × 2506
    obs: 'nCount_RNA', 'nFeature_RNA', 'sample', 'nCount_HTO', 'nFeature_HTO', 'HTO_maxID', 'HTO_secondID', 'HTO_margin', 'HTO_classification', 'hash.ID', 'RNA_snn_res.0.6', 'seurat_clusters', 'percent_mt', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'rnaseq_labels', 'immgen_labels', 'batch', '_scvi_batch', '_scvi_labels', 'leiden', 'pMito', 'nGenes', 'nCounts', 'pass_basic_filter', 'Size_Factor', 'initial_cell_size', 'counts_Size_Factor', 'initial_counts_cell_size', 'scVI_normalized_Size_Factor', 'initial_scVI_normalized_cell_size', 'cell_cycle_phase'
    var: 'nCells', 'nCounts', 'pass_basic_filter', 'log_cv', 'score', 'log_m', 'frac', 'use_for_pca'
    uns: 'HTO_maxID_colors', '_scvi_manager_uuid', '_scvi_uuid', 'batch_colors', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'umap', 'pp', 'velocyto_SVR', 'PCs', 'explained_variance_ratio_', 'pca_mean', 'pca_fit', 'feature_selection', 'cell_phase_genes'
    obsm: 'X_mde', 'X_pca', '

In [18]:
type(hvg)

anndata._core.anndata.AnnData

In [19]:
del(hvg.uns["cell_phase_genes"])

In [20]:
hvg.write_h5ad("../data/h5ad/hvg_integrated_170824_cell_cycle_phases.h5ad")